# Machine learning and deep learning methods applied to predicting customer status

Purpose: Predict active and not active clients based on the proposed data structure and naive data structure. Here, we implemented Support Vector Machines (SVM), Random Forest (RF), K-nearest Neighbours (KNN) and Lasso.

Author: Gabriel Rodrigues Palma and Rafael de Andrade Moral

# Packages used in the project

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import LeakyReLU

# Machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Functions used in the project

In [2]:
def create_dataset(path, response_class):
    ''' This function reads and prepare the datasets for applying the ML and DL methods'''
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0', 'Subject'])
    explanatory_variables = data.drop(columns = response_class)
    response_variables = data[response_class]        
    binarizer = LabelBinarizer()
    response_variables = binarizer.fit_transform(response_variables)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    hot_encode_response_variable = onehot_encoder.fit_transform(np.array(response_variables).reshape(-1, 1))          
    
    return(explanatory_variables, 
           hot_encode_response_variable, response_variables)

def check_zero_division_and_get_rates(cm):
    ''' This functions checks for divisions per zeros in the computation of 
       True and false positive rate based on the confusion matrix array. Also, 
       this function returns the checked rates'''
    
    if any(np.sum(cm, axis = 1)==0):            
            tpr = cm[:,1][1]/(np.sum(cm, axis = 1)[1]+1e-16)
            fpr = cm[:,1][0]/(np.sum(cm, axis = 1)[0]+1e-16)    
    else:           
        rates = cm[:,1]/np.sum(cm, axis = 1)    
        fpr = rates[0]    
        tpr = rates[1]
    if np.isnan(tpr):        
        tpr=0
        
    return(tpr, fpr)

def check_and_compute_rates(predictions, 
                            classes, 
                            cm):    
    ''' This function returns the values of true and false positive rate for special cases
       where the division is not possible to obtain automatically based on the confusion
       matrix array provided by sklearn'''
        
    if (sum(predictions) == 0 and sum(classes) == 0):        
        fpr = 0
        tpr = 0
    elif (sum(predictions) == len(predictions) and sum(classes) == len(classes)):        
        fpr = 0
        tpr = 1
    else:            
        tpr, fpr = check_zero_division_and_get_rates(cm)
        
    return(tpr, fpr)
    
def get_rates(y_pred,
              y_true):
    ''' This function get the true and false positive rates based on the 
       predictied'''
            
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    tpr, fpr = check_and_compute_rates(y_pred, y_true, cm)
        
    return(tpr, fpr)

def get_rates_by_cross_validation(raw_data):
    ''' This function obtains the accuracy, true and false positive rates 
       based on the cross k-fold cross validation'''
            
    for train_index, test_index in KFold(n_splits=5, shuffle=True).split(patterns):
        
        x_train, x_test = patterns[train_index], patterns[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
                
        pbp_predictions = pbp_prediction(patterns_array=x_test, 
                                         clustered_patterns = clustered_patterns, 
                                         d_base = d_base, alpha = alpha, 
                   outbreak_p_means = prediction.obtain_p_means_with_distance,
                   outbreak_prediction = prediction.predict_with_distance)
        rates = get_rates(predictions = pbp_predictions, classes = y_test)
        tpr.append(rates[0])
        fpr.append(rates[1])
    
    return(np.mean(tpr[tpr!=np.nan]), np.mean(fpr[fpr!=np.nan]))

def get_statistics(y_pred, y_true):
    ''' This function obtains the following statistics: Accuracy, True positive rate and 
       False positive rate'''

    auc = roc_auc_score(y_score = y_pred,
              y_true = y_true)
    acc = accuracy_score(y_pred = y_pred,
                         y_true = y_true)
    rates = get_rates(y_pred = y_pred,
              y_true = y_true)
    tpr = rates[0]
    fpr = rates[1]
    return([acc, tpr, fpr, auc])

# def get_methods_performance(X_train, X_test, 
#                             y_train, y_test, 
#                             raw_data):
#     ''' This function obtain the performance of each selected model'''
#     kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
#     # Support Vector Machine -----
#     ## Non-linear
#     nonlinear_svm = svm.NuSVC(gamma="auto")
#     # Create the parameter space
#     params = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
#               "gamma": ['auto', 'scale'],
#              "degree": np.arange(1, 10, 1)}
#     nonlinear_svm_cv = RandomizedSearchCV(nonlinear_svm, params, cv=kf)
#     nonlinear_svm_cv.fit(X_train, y_train)
#     nonlinear_svm_predictions = nonlinear_svm_cv.predict(X_test)
    
#     ## Polinomial
#     polinomial_svm = svm.SVC(kernel='poly', degree = 3)
#     polinomial_svm.fit(X_train, y_train)
#     polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
#     # Random forest -----
#     rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
#     rf.fit(X_train, y_train)
#     random_forest_predictions = rf.predict(X_test)
    
#     # KNN -----
#     knn = KNeighborsClassifier(n_neighbors=1)
#     knn.fit(X_train, y_train)
#     knn_predictions = knn.predict(X_test)
    
#     # Lasso -----
#     lasso = Lasso(alpha=0.02)
#     lasso.fit(X_train, y_train)
#     lasso_predictions = np.round(lasso.predict(X_test))
    
#     # Deep Neural Network -----
# #     if raw_data==True:
# #         dnn = keras.models.load_model('DNN/DNN_model_params.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     elif raw_data == 'combined':
# #         dnn = keras.models.load_model('DNN/DNN_model_fulldata.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     else:    
# #         dnn = keras.models.load_model('DNN/DNN_model_hmm.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))

    
#     # Obtaining statistics -----
#     polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
#                                         y_true = y_test)
#     nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
#                                        y_true = y_test)
#     random_forest_performance = get_statistics(y_pred = random_forest_predictions,
#                                        y_true = y_test)
#     knn_performance = get_statistics(y_pred = knn_predictions,
#                                        y_true = y_test)    
#     lasso_performance = get_statistics(y_pred = lasso_predictions,
#                                        y_true = y_test)
# #     dnn_performance = get_statistics(y_pred = dnn_predictions,
# #                                        y_true = y_test)

#     return(polynomial_svm_performance, nonlinear_svm_performance, 
#            random_forest_performance, knn_performance, 
#            lasso_performance)
    
# def get_results_data(explanatory_variables,                             
#                      response_variables,                                                         
#                      test_size, raw_data):
#     ''' This function obtain the performance of the ML and DL methods based on
#        the prediction of client status'''
#     X_train, X_test, y_train, y_test = train_test_split(explanatory_variables, 
#                                                         response_variables, 
#                                                         test_size = test_size, 
#                                                         random_state = 42)
   
#     # Obtaining methods performance
#     polynomial_svm_performance, nonlinear_svm_performance, \
#     random_forest_performance, knn_performance, \
#     lasso_performance = get_methods_performance(X_train, X_test, 
#                                               y_train, y_test, 
#                                               raw_data)
        
    
#     model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
#                                                             'True Positive Rate', 
#                                                             'False Positive Rate', 
#                                                             'AUROC'],
#                                               'Polinomial SVM': [polynomial_svm_performance[0], 
#                                                                  polynomial_svm_performance[1], 
#                                                                  polynomial_svm_performance[2],
#                                                                 polynomial_svm_performance[3]], 
#                                               'Non linear SVM': [nonlinear_svm_performance[0], 
#                                                                  nonlinear_svm_performance[1], 
#                                                                  nonlinear_svm_performance[2], 
#                                                                 nonlinear_svm_performance[3]], 
#                                               'Random Forest': [random_forest_performance[0], 
#                                                                 random_forest_performance[1], 
#                                                                 random_forest_performance[2], 
#                                                                random_forest_performance[3]], 
#                                               'KNN': [knn_performance[0], 
#                                                       knn_performance[1], 
#                                                       knn_performance[2], 
#                                                       knn_performance[3]],                                                
#                                               'Lasso': [lasso_performance[0], 
#                                                         lasso_performance[1], 
#                                                         lasso_performance[2], 
#                                                         lasso_performance[3]]})

#     return(model_outputs_performance)
    

In [3]:
def get_methods_predictions(X_train, X_test, 
                            y_train, y_test, 
                            data_type):
    ''' This function obtain the performance of each selected model'''
    kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    # Support Vector Machine -----
    ## Non-linear
    nonlinear_svm = svm.NuSVC()
    
    nonlinear_svm.fit(X_train, y_train)
    
    nonlinear_svm_predictions = nonlinear_svm.predict(X_test)
    
    ## Polinomial
    polinomial_svm = svm.SVC(kernel='poly', degree = 3)
    polinomial_svm.fit(X_train, y_train)
    polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
    # Random forest -----
    rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
    rf.fit(X_train, y_train)
    random_forest_predictions = rf.predict(X_test)
    
    # KNN -----
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    
    # Lasso -----
    lasso = Lasso(alpha=0.02)
    lasso.fit(X_train, y_train)
    lasso_predictions = np.round(lasso.predict(X_test))
    
    # Deep Neural Network -----
    if data_type == 'All':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(46,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm2':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(9,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))

    elif data_type == 'm3':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(12,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm4':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(15,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
        
    else:    
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(18,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
    

    return(polinomial_svm_predictions, nonlinear_svm_predictions, 
           random_forest_predictions, knn_predictions, 
           lasso_predictions, dnn_predictions)
def get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                                random_forest_predictions, knn_predictions, 
                                lasso_predictions, dnn_predictions, 
                                reponse_variable):
    
    '''This function obtain the methods performance based on their predictions for the leave one out cross valition'''
    polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
                                    y_true = reponse_variable)
    nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
                                       y_true = reponse_variable)
    random_forest_performance = get_statistics(y_pred = random_forest_predictions,
                                       y_true = reponse_variable)
    knn_performance = get_statistics(y_pred = knn_predictions,
                                       y_true = reponse_variable)    
    lasso_performance = get_statistics(y_pred = lasso_predictions,
                                       y_true = reponse_variable)
    dnn_performance = get_statistics(y_pred = dnn_predictions,
                                        y_true = reponse_variable)
    return(polynomial_svm_performance, nonlinear_svm_performance, 
           random_forest_performance, knn_performance, 
           lasso_performance, dnn_performance)

    
def get_results_data_loo(explanatory_variables,                         
                         response_variables,                                                         
                         data_type):
    ''' This function obtain the performance of the ML and DL methods based on
       the prediction of client status based on leave one out cross validation'''

    loo = LeaveOneOut()
    loo.get_n_splits(explanatory_variables)
    polinomial_svm_predictions, nonlinear_svm_predictions, \
           random_forest_predictions, knn_predictions, \
           lasso_predictions, dnn_predictions = ([], [], [], [], [], [])
    
    for train_index, test_index in loo.split(explanatory_variables):
        X_train, X_test = explanatory_variables[train_index], explanatory_variables[test_index]
        y_train, y_test = response_variables[train_index], response_variables[test_index]                
             
        ## Correcting errors related to column vector (n_observations, ) to (, n_observations)
        y_train = y_train.ravel() 
        y_test = y_test.ravel()        
    
        polynomial_svm_prediction, nonlinear_svm_prediction, \
        random_forest_prediction, knn_prediction, \
        lasso_prediction, dnn_prediction = get_methods_predictions(X_train, X_test, 
                                                   y_train, y_test, 
                                                   data_type)
        
        polinomial_svm_predictions.append(polynomial_svm_prediction[0])
        nonlinear_svm_predictions.append(nonlinear_svm_prediction[0])
        random_forest_predictions.append(random_forest_prediction[0])
        knn_predictions.append(knn_prediction[0])        
        lasso_predictions.append(lasso_prediction[0]) 
        dnn_predictions.append(dnn_prediction[0])
 
    polynomial_svm_performance, nonlinear_svm_performance, \
       random_forest_performance, knn_performance, \
       lasso_performance, dnn_performance = get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                            random_forest_predictions, knn_predictions, 
                            lasso_predictions, dnn_predictions, 
                            list(response_variables.flatten()))
    
    model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
                                                            'True Positive Rate', 
                                                            'False Positive Rate', 
                                                            'AUROC'],
                                              'Polinomial SVM': [polynomial_svm_performance[0], 
                                                                 polynomial_svm_performance[1], 
                                                                 polynomial_svm_performance[2],
                                                                polynomial_svm_performance[3]], 
                                              'Non linear SVM': [nonlinear_svm_performance[0], 
                                                                 nonlinear_svm_performance[1], 
                                                                 nonlinear_svm_performance[2], 
                                                                nonlinear_svm_performance[3]], 
                                              'Random Forest': [random_forest_performance[0], 
                                                                random_forest_performance[1], 
                                                                random_forest_performance[2], 
                                                               random_forest_performance[3]], 
                                              'KNN': [knn_performance[0], 
                                                      knn_performance[1], 
                                                      knn_performance[2], 
                                                      knn_performance[3]],                                                
                                              'Lasso': [lasso_performance[0], 
                                                        lasso_performance[1], 
                                                        lasso_performance[2], 
                                                        lasso_performance[3]], 
                                             'DNN': [dnn_performance[0], 
                                                        dnn_performance[1], 
                                                        dnn_performance[2], 
                                                        dnn_performance[3]]})

    return(model_outputs_performance)

# Importing Datasets

Before obtaining the performance of the learning algorithms selected for this paper, we need to import the datasets related to the Hidden Markov Models features and the peak features.

## Zscored scale Midline EEG data Trial 12

In [4]:
all_Zscored_scale_explanatory_variables_t1, \
  all_Zscored_scale_hot_encode_response_variable_t1, all_Zscored_scale_response_variable_t1 = create_dataset(path = '../../output_data/New_data/SplitDatasets/ZscoredScaledTrial1.csv', 
                                                                                           response_class = 'features_class')

In [5]:
all_Zscored_scale_explanatory_variables_t1.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.1,hmm5_sd.2,hmm5_sd.3,hmm5_sd.4,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5
0,0.430807,-9.240701,0.899261,-0.002411,0.348,0.216,0.065,0.037,0.515900,0.484100,...,0.011,0.058,0.016,0.024,0.012,0.180843,0.240996,0.243103,0.144828,0.190230
1,0.379726,-28.295271,0.994679,-0.002637,0.209,0.364,0.048,0.092,0.530876,0.469124,...,0.112,0.016,0.023,0.026,0.015,0.156503,0.214782,0.282709,0.138557,0.207449
2,-0.190530,30.395558,0.096708,-0.002210,0.269,0.147,0.064,0.032,0.404959,0.595041,...,0.018,0.016,0.060,0.010,0.009,0.229291,0.181818,0.179896,0.210456,0.198539
3,0.018348,11.094229,0.167587,-0.001766,0.264,0.155,0.045,0.034,0.440678,0.559322,...,0.011,0.036,0.014,0.009,0.022,0.190485,0.208398,0.214176,0.224576,0.162365
4,0.738628,-31.642053,1.505026,-0.002939,0.275,0.459,0.053,0.082,0.627565,0.372435,...,0.024,0.075,0.028,0.015,0.014,0.237708,0.252226,0.111692,0.255517,0.142857


## Zscored scale Midline EEG data Trial 12

In [6]:
all_Zscored_scale_explanatory_variables_t12, \
  all_Zscored_scale_hot_encode_response_variable_t12, all_Zscored_scale_response_variable_t12 = create_dataset(path = '../../output_data/New_data/SplitDatasets/ZscoredScaledTrial12.csv', 
                                                                                           response_class = 'features_class')

In [7]:
all_Zscored_scale_explanatory_variables_t12.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.1,hmm5_sd.2,hmm5_sd.3,hmm5_sd.4,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5
0,0.505831,3.734294,1.254882,-0.003140,0.394,0.253,0.073,0.048,0.315457,0.684543,...,0.017,0.008,0.028,0.014,0.067,0.194532,0.219769,0.256572,0.170347,0.158780
1,0.154855,-90.618525,1.082626,-0.002968,0.224,0.442,0.029,0.149,0.590447,0.409553,...,0.010,0.014,0.008,0.044,0.153,0.149390,0.257114,0.183943,0.295732,0.113821
2,-0.381763,-108.664622,-0.051024,-0.001874,0.126,0.261,0.022,0.090,0.470644,0.529356,...,0.016,0.004,0.007,0.026,0.091,0.129735,0.188447,0.133523,0.357008,0.191288
3,-0.024115,-51.478291,0.385727,-0.002211,0.308,0.171,0.059,0.031,0.423729,0.576271,...,0.006,0.013,0.009,0.026,0.048,0.095450,0.329170,0.116860,0.266726,0.191793
4,0.289524,-99.254789,1.295104,-0.003043,0.418,0.236,0.080,0.059,0.412071,0.587929,...,0.022,0.026,0.007,0.011,0.068,0.262227,0.234131,0.123829,0.138398,0.241415


# Obtaining learning algorithms performance based on Leave one out cross validation

Now, we are obtaining accuracy, trus and false positive rates for Polinomial and no linear Support Vector Machines, Random Forests, K-neirest neighbors and Lasso learning algorithms.

## Zscored scaled EEG data Trial 1

For the raw Midline EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [16]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1),                                                  
                     response_variables = all_Zscored_scale_response_variable_t1,                                                                           
                     data_type = 'All')

2023-04-04 15:32:59.231352: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-04 15:32:59.232902: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


2023-04-04 15:32:59.728172: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-04 15:33:00.544436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 155ms/step


2023-04-04 15:33:09.885086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:33:11.185076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 177ms/step


2023-04-04 15:41:25.470178: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:41:26.870739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 188ms/step


2023-04-04 15:41:37.241884: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.489362,0.382979,0.425532,0.404255,0.382979
1,True Positive Rate,0.000000,0.500000,0.318182,0.318182,0.000000,0.318182
2,False Positive Rate,0.000000,0.520000,0.560000,0.480000,0.240000,0.560000
3,AUROC,0.500000,0.490000,0.379091,0.419091,0.380000,0.379091


### The features for HMM with 2 parameters

In [17]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm2_mean.1', 
                    'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                    'hmm2_p1']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_Zscored_scale_explanatory_variables_t1.columns]

In [18]:
all_Zscored_scale_explanatory_variables_t1_m2 = all_Zscored_scale_explanatory_variables_t1[selected_columns_m2]

In [19]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1_m2),                                                  
                     response_variables = all_Zscored_scale_response_variable_t1,                                                                           
                     data_type = 'm2')

2023-04-04 15:41:39.926237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 209ms/step


2023-04-04 15:41:50.907764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:41:52.587335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 178ms/step


2023-04-04 15:42:03.453275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:42:04.779941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 261ms/step


2023-04-04 15:52:09.058840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 245ms/step


2023-04-04 15:52:22.937447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.468085,0.468085,0.425532,0.510638,0.255319
1,True Positive Rate,0.000000,0.590909,0.500000,0.318182,0.000000,0.045455
2,False Positive Rate,0.000000,0.640000,0.560000,0.480000,0.040000,0.560000
3,AUROC,0.500000,0.475455,0.470000,0.419091,0.480000,0.242727


### The features for HMM with 3 parameters

In [20]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_Zscored_scale_explanatory_variables_t1.columns]

In [21]:
all_Zscored_scale_explanatory_variables_t1_m3 = all_Zscored_scale_explanatory_variables_t1[selected_columns_m3]

In [22]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1_m3),                                                  
                     response_variables = all_Zscored_scale_response_variable_t1,                                                                           
                     data_type = 'm3')

2023-04-04 15:52:25.651002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:52:39.097486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 344ms/step


2023-04-04 15:52:40.690543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:52:54.358146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 278ms/step


2023-04-04 15:52:56.407258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 15:53:09.281226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 16:05:23.700785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:05:38.931482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 392ms/step


2023-04-04 16:05:40.571209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:05:55.453274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 432ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.340426,0.425532,0.425532,0.510638,0.297872
1,True Positive Rate,0.000000,0.409091,0.318182,0.318182,0.000000,0.181818
2,False Positive Rate,0.000000,0.720000,0.480000,0.480000,0.040000,0.600000
3,AUROC,0.500000,0.344545,0.419091,0.419091,0.480000,0.290909


### The features for HMM with 4 parameters

In [8]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3']
selected_columns_m4_array = [elem in selected_columns_m4 for elem in all_Zscored_scale_explanatory_variables_t1.columns]

In [9]:
all_Zscored_scale_explanatory_variables_t1_m4 = all_Zscored_scale_explanatory_variables_t1[selected_columns_m4]

In [10]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1_m4),                                               
                     response_variables = all_Zscored_scale_response_variable_t1,                                                                           
                     data_type = 'm4')

Metal device set to: Apple M1


2023-04-06 17:50:58.423409: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 17:50:58.423670: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-06 17:50:58.724669: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-06 17:50:59.308450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 92ms/step


2023-04-06 17:51:05.126347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 17:51:06.070905: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 61ms/step


...

1/1 [==============================] - 0s 163ms/step


2023-04-06 17:57:32.280904: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 17:57:34.105959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 141ms/step


2023-04-06 17:57:43.197093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.531915,0.531915,0.425532,0.404255,0.319149
1,True Positive Rate,0.000000,0.590909,0.409091,0.318182,0.000000,0.318182
2,False Positive Rate,0.000000,0.520000,0.360000,0.480000,0.240000,0.680000
3,AUROC,0.500000,0.535455,0.524545,0.419091,0.380000,0.319091


### The features for HMM with 5 parameters

In [26]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Zscored_scale_explanatory_variables_t1.columns]

In [27]:
all_Zscored_scale_explanatory_variables_t1_m5 = all_Zscored_scale_explanatory_variables_t1[selected_columns_m5]

In [28]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t1_m5),                                                  
                     response_variables = all_Zscored_scale_response_variable_t1,                                                                           
                     data_type = 'm5')

2023-04-04 16:19:41.202847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:19:56.715544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 390ms/step


2023-04-04 16:19:58.609747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:20:13.395014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 346ms/step


2023-04-04 16:20:14.716880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:20:29.710371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 16:34:32.324219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:34:50.295889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 511ms/step


2023-04-04 16:34:51.775828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:35:09.891817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 538ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.468085,0.319149,0.425532,0.510638,0.340426
1,True Positive Rate,0.000000,0.500000,0.227273,0.318182,0.000000,0.227273
2,False Positive Rate,0.000000,0.560000,0.600000,0.480000,0.040000,0.560000
3,AUROC,0.500000,0.470000,0.313636,0.419091,0.480000,0.333636


## Zscored scaled Midline EEG data Trial 12

For the Minmax scale EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [8]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12),                                                  
                     response_variables = all_Zscored_scale_response_variable_t12,                                                                           
                     data_type = 'All')

2023-04-04 16:49:21.912541: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-04 16:49:21.912795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


2023-04-04 16:49:22.182511: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-04 16:49:22.869049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 103ms/step


2023-04-04 16:49:29.525025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:49:30.512336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 102ms/step


2023-04-04 16:55:41.426497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:55:42.458456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 131ms/step


2023-04-04 16:55:50.055916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.765957,0.531915,0.638298,0.340426,0.787234
1,True Positive Rate,0.000000,0.909091,0.363636,0.681818,0.000000,0.818182
2,False Positive Rate,0.000000,0.360000,0.320000,0.400000,0.360000,0.240000
3,AUROC,0.500000,0.774545,0.521818,0.640909,0.320000,0.789091


### The features for HMM with 2 parameters

In [9]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm2_mean.1', 
                    'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                    'hmm2_p1']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_Zscored_scale_explanatory_variables_t12.columns]

In [10]:
all_Zscored_scale_explanatory_variables_t12_m2 = all_Zscored_scale_explanatory_variables_t12[selected_columns_m2]

In [11]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12_m2),                                                 
                     response_variables = all_Zscored_scale_response_variable_t12,                                                                           
                     data_type = 'm2')

2023-04-04 16:55:51.700216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 147ms/step


2023-04-04 16:56:00.148593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:56:01.219558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 149ms/step


2023-04-04 16:56:09.278001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 16:56:10.769070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 17:04:05.254200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:04:07.193163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 222ms/step


2023-04-04 17:04:17.524710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:04:19.056871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:04:29.658710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 290ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.787234,0.531915,0.638298,0.340426,0.808511
1,True Positive Rate,0.000000,0.909091,0.681818,0.681818,0.000000,0.818182
2,False Positive Rate,0.000000,0.320000,0.600000,0.400000,0.360000,0.200000
3,AUROC,0.500000,0.794545,0.540909,0.640909,0.320000,0.809091


### The features for HMM with 3 parameters

In [12]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_Zscored_scale_explanatory_variables_t12.columns]

In [13]:
all_Zscored_scale_explanatory_variables_t12_m3 = all_Zscored_scale_explanatory_variables_t12[selected_columns_m3]

In [14]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12_m3),                                                
                     response_variables = all_Zscored_scale_response_variable_t12,                                                                           
                     data_type = 'm3')

2023-04-04 17:04:32.063611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 236ms/step


2023-04-04 17:04:42.860733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:04:44.227514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:04:55.597038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 290ms/step


2023-04-04 17:04:57.498290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:05:08.420119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 17:15:21.376973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:15:35.333002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 350ms/step


2023-04-04 17:15:37.494717: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:15:51.119441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 344ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.744681,0.574468,0.617021,0.340426,0.829787
1,True Positive Rate,0.000000,0.909091,0.500000,0.636364,0.000000,0.772727
2,False Positive Rate,0.000000,0.400000,0.360000,0.400000,0.360000,0.120000
3,AUROC,0.500000,0.754545,0.570000,0.618182,0.320000,0.826364


### The features for HMM with 4 parameters

In [15]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3']
selected_columns_m4_array = [elem in selected_columns_m4 for elem in all_Zscored_scale_explanatory_variables_t12.columns]

In [16]:
all_Zscored_scale_explanatory_variables_t12_m4 = all_Zscored_scale_explanatory_variables_t12[selected_columns_m4]

In [17]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12_m4),                                                  
                     response_variables = all_Zscored_scale_response_variable_t12,                                                                           
                     data_type = 'm4')

2023-04-04 17:15:52.978880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:16:06.496098: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 259ms/step


2023-04-04 17:16:08.158246: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:16:18.834824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 254ms/step


2023-04-04 17:16:20.063598: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:16:29.983978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 17:26:05.204780: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:26:16.871667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 322ms/step


2023-04-04 17:26:18.126079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:26:29.771295: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 410ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.723404,0.638298,0.617021,0.340426,0.851064
1,True Positive Rate,0.000000,0.863636,0.590909,0.636364,0.000000,0.909091
2,False Positive Rate,0.000000,0.400000,0.320000,0.400000,0.360000,0.200000
3,AUROC,0.500000,0.731818,0.635455,0.618182,0.320000,0.854545


### The features for HMM with 5 parameters

In [18]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Zscored_scale_explanatory_variables_t12.columns]

In [19]:
all_Zscored_scale_explanatory_variables_t12_m5 = all_Zscored_scale_explanatory_variables_t12[selected_columns_m5]

In [20]:
get_results_data_loo(explanatory_variables = np.array(all_Zscored_scale_explanatory_variables_t12_m5),                                                  
                     response_variables = all_Zscored_scale_response_variable_t12,                                                                           
                     data_type = 'm5')

2023-04-04 17:26:31.742515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:26:44.543459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 277ms/step


2023-04-04 17:26:45.713334: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:26:58.681147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 412ms/step


2023-04-04 17:27:00.351301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:27:13.269445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-04 17:38:35.080891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:38:49.812455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 394ms/step


2023-04-04 17:38:51.435364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-04 17:39:07.579706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 458ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.531915,0.787234,0.617021,0.595745,0.340426,0.851064
1,True Positive Rate,0.000000,0.909091,0.545455,0.590909,0.000000,0.863636
2,False Positive Rate,0.000000,0.320000,0.320000,0.400000,0.360000,0.160000
3,AUROC,0.500000,0.794545,0.612727,0.595455,0.320000,0.851818
